# Erweiterte Konzepte in Python

## Dataclasses

Dokumentation: [Dataclasses](https://docs.python.org/3/library/dataclasses.html)

Dataclasses sind eine spezielle Art von Klassen in Python, die hauptsächlich zur Speicherung von Daten entwickelt wurden. Sie wurden in Python 3.7 eingeführt und bieten eine elegante Möglichkeit, Klassen zu erstellen, die primär dazu dienen, strukturierte Daten zu halten.
Der große Vorteil von Dataclasses liegt darin, dass Python automatisch viele häufig benötigte Methoden für uns generiert, die wir sonst manuell schreiben müssten. Stellen Sie sich vor, Sie müssten für jede Datenklasse immer wieder die gleichen `__init__`, `__repr__` und `__eq__` Methoden schreiben - das wäre ziemlich mühsam und fehleranfällig.

In [ ]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    age: int
    email: str

# Dataclass automatically generates __init__, __repr__, and __eq__ methods
p1 = Person(name="Alice", age=30, email="alice@gmail.com")
p2 = Person(name="Bob", age=25, email="bob@gmail.com")



In [ ]:


print(p1)
print(p2)

print(p1.name)
print(p1.age)
print(p1.email)

print(f"{p1 == p2 = }")
print(f"{p1 == p1 = }")

p1.name = "Alice Smith"
print(p1)


Dataclasses bieten jedoch weit mehr Flexibilität als die einfache Grundfunktionalität. Mit zusätzlichen Parametern und speziellen Funktionen können wir das Verhalten unserer Datenklassen präzise steuern.

Der Parameter `order=True` ermöglicht es, dass Python automatisch Vergleichsoperatoren (`<`, `>`, `<=`, `>=`) für unsere Klasse generiert. Dadurch können wir Instanzen der Klasse sortieren. Mit `frozen=True` machen wir die Dataclass unveränderlich (immutable) - nach der Erstellung können die Attribute nicht mehr verändert werden.

Die `field()` Funktion gibt uns feine Kontrolle über einzelne Attribute. Mit `compare=False` können wir bestimmte Felder von Vergleichsoperationen ausschließen, `default=None` setzt Standardwerte, und `repr=False` verhindert, dass ein Attribut in der String-Darstellung erscheint.

Besonders mächtig ist die `__post_init__` Methode, die automatisch nach der Initialisierung aufgerufen wird. Hier können wir Validierungen durchführen, berechnete Werte setzen oder andere Nachbearbeitungen vornehmen. Bei frozen Dataclasses müssen wir `object.__setattr__()` verwenden, um Attribute zu ändern.

In [ ]:
from dataclasses import dataclass, field
from typing import Optional

@dataclass(order=True, frozen=True)
class Person:
    name: str = field(compare=False)
    age: int
    email: Optional[str] = field(compare=False, default=None, repr=False)
    
    def __post_init__(self):
        if self.age < 0:
            raise ValueError("Age cannot be negative")
        
        if not self.email:
            # self.email = "No email provided"                      # This will raise an error because the dataclass is frozen
            object.__setattr__(self, 'email', "No email provided")  # This is how you can set a default value for a frozen dataclass
            # The Better way to handle this is to use a default value in the field definition
            
        print(f"Created Person: {self.name}, Age: {self.age}, Email: {self.email}")
        
    # You can add methods to the dataclass
    @property
    def is_adult(self) -> bool:
        return self.age >= 18
    
    
persons = (
    Person(name="Alice", age=12),
    Person(name="Bob", age=45, email="bobthemaster@gmx.de"),
    Person(name="Charlie", age=18, email="charlie@gmail.com"),
    Person(name="Alice", age=32),
    Person(name="Tim", age=10)
)



In [ ]:

print("\n"*3)
print("Persons sorted by age:")
for person in sorted(persons):
    print(person)
    

print("\n"*3)
print("Only adults:")
print([person for person in persons if person.is_adult])

#persons[0].name = "Alice Smith"  # This will raise an error because the dataclass is frozen

### Warum Dataclasses verwenden?
Dataclasses bieten mehrere wichtige Vorteile gegenüber herkömmlichen Klassen:
- Weniger Code schreiben: Sie müssen nicht immer wieder die gleichen Standardmethoden implementieren.
- Bessere Lesbarkeit: Der Code wird klarer und fokussiert sich auf die eigentlichen Daten.
- Typsicherheit: Durch die Verwendung von Type Hints wird der Code robuster und besser dokumentiert.
- Automatische Funktionalität: Vergleiche, String-Repräsentationen und Hashing funktionieren automatisch.

### Wann sollten Sie Dataclasses verwenden?
Dataclasses sind ideal für Situationen, in denen Sie:

- Strukturierte Daten speichern möchten
- Klassen benötigen, die hauptsächlich als Datencontainer fungieren
- Schnell und sauber Datenstrukturen definieren möchten
- Von automatisch generierten Methoden profitieren möchten
- Sie sind weniger geeignet für komplexe Klassen mit viel Geschäftslogik oder wenn Sie vollständige Kontrolle über die Implementierung aller Methoden benötigen.

### Zusammenfassung
Dataclasses sind ein mächtiges Werkzeug in Python, das die Erstellung von datenorientierten Klassen erheblich vereinfacht. Sie reduzieren Boilerplate-Code, verbessern die Lesbarkeit und bieten gleichzeitig robuste Funktionalität. Für die meisten Anwendungsfälle, in denen Sie strukturierte Daten verwalten müssen, sind Dataclasses die eleganteste Lösung.



## Enums

Dokumentation: [Enums](https://docs.python.org/3/library/enum.html)

Enums (Enumerationen) sind eine spezielle Klasse in Python, die es ermöglicht, eine Gruppe von konstanten Werten zu definieren. Sie sind besonders nützlich, wenn Sie eine feste Menge von Werten benötigen, die klar benannt und voneinander unterschieden werden können. Enums verbessern die Lesbarkeit des Codes und verhindern Fehler, die durch die Verwendung von "magischen Zahlen" oder Strings entstehen könnten.


In [ ]:
def buy_shirt(color: str, size: str):
    valid_colors = ['red', 'green', 'blue']
    valid_sizes = ['S', 'M', 'L', 'XL']

    if color not in valid_colors:
        raise ValueError(f"Invalid color: {color}. Valid options are: {valid_colors}")

    if size not in valid_sizes:
        raise ValueError(f"Invalid size: {size}. Valid options are: {valid_sizes}")

    print(f"Buying a {color} shirt in size {size}.")



In [ ]:

try:
    buy_shirt('red', 'M')  # Gültiger Aufruf
    buy_shirt('yellow', 'M')  # Ungültiger Aufruf
except ValueError as e:
    print(e)

Die Verwendung von einfachen Strings für Parameter wie `color` und `size` bringt mehrere Probleme mit sich. Erstens gibt es keine Typsicherheit - der Entwickler könnte versehentlich `'Red'` statt `'red'` schreiben oder sich bei der Groß-/Kleinschreibung vertun. Zweitens sind die gültigen Werte nicht sofort ersichtlich, ohne den Code zu lesen oder die Dokumentation zu konsultieren. Drittens bieten moderne IDEs keine Autovervollständigung für diese String-Werte an, was zu Tippfehlern führen kann.

Darüber hinaus ist die Wartung problematisch: Wenn neue Farben oder Größen hinzugefügt werden sollen, müssen die Listen im Code aktualisiert werden, und es besteht die Gefahr, dass diese an verschiedenen Stellen im Code inkonsistent werden. Bei größeren Projekten können solche "magischen Strings" schnell zu schwer auffindbaren Fehlern führen.

Enum-Klassen lösen diese Probleme elegant: Sie definieren eine feste Menge von benannten Konstanten, die typsicher sind und von der IDE unterstützt werden. Ungültige Werte werden bereits zur Laufzeit abgefangen, und die verfügbaren Optionen sind klar definiert und dokumentiert.

In [ ]:
from enum import Enum

class Color(Enum):
    RED = 'red'
    GREEN = 'green'
    BLUE = 'blue'

class Size(Enum):
    S = 'S'
    M = 'M'
    L = 'L'
    XL = 'XL'

def buy_shirt_enum(color: Color, size: Size):
    print(f"Buying a {color.value} shirt in size {size.value}.")



In [ ]:

try:
    buy_shirt_enum(Color.RED, Size.M)  # Gültiger Aufruf
    buy_shirt_enum(Color('yellow'), Size.M)  # Ungültiger Aufruf
except ValueError as e:
    print(e)

Neben den einfachen Enums gibt es auch spezialisierte Varianten für bestimmte Anwendungsfälle. Eine besonders mächtige Variante ist das `Flag` Enum, das für Situationen entwickelt wurde, in denen mehrere Werte gleichzeitig aktiv sein können - wie bei Berechtigungen, Zuständen oder Optionen.

Das `Flag` Enum unterscheidet sich von normalen Enums dadurch, dass seine Werte mit bitweisen Operatoren (`|`, `&`, `^`) kombiniert werden können. Jeder Enum-Wert repräsentiert eine Zweierpotenz (1, 2, 4, 8, ...), wodurch sie sich perfekt für Bitmasken eignen.

Die `auto()` Funktion ist dabei besonders praktisch: Sie weist automatisch passende Werte zu, ohne dass wir uns Gedanken über die konkrete Zahlenzuweisung machen müssen. Bei Flag Enums sorgt `auto()` dafür, dass jeder Wert eine eindeutige Zweierpotenz erhält, was für die bitweise Verknüpfung erforderlich ist.

Diese Kombination aus Flag Enum und `auto()` ermöglicht es, elegante und typsichere Lösungen für komplexe Berechtigungssysteme oder Konfigurationsoptionen zu erstellen, bei denen mehrere Eigenschaften gleichzeitig aktiv sein können.

In [ ]:
from enum import Flag, auto

class Berechtigung(Flag):
    LESEN = auto()
    SCHREIBEN = auto()
    AUSFUEHREN = auto()
    LOESCHEN = auto()

# Berechtigungen können kombiniert werden:
user_rechte = Berechtigung.LESEN | Berechtigung.SCHREIBEN
admin_rechte = Berechtigung.LESEN | Berechtigung.SCHREIBEN | Berechtigung.AUSFUEHREN | Berechtigung.LOESCHEN


print(f"{Berechtigung.LESEN = }")
print(f"{Berechtigung.SCHREIBEN = }")
print(f"{Berechtigung.AUSFUEHREN = }")
print(f"{Berechtigung.LOESCHEN = }")
print(f"{user_rechte = }")  # Berechtigung.LESEN | Berechtigung.SCHREIBEN
print(f"{admin_rechte = }")  # Berechtigung.LESEN | Berechtigung.SCHREIBEN | Berechtigung.AUSFUEHREN | Berechtigung.LOESCHEN




In [ ]:

# Überprüfung von Berechtigungen:
def kann_datei_bearbeiten(rechte: Berechtigung) -> bool:
    """Prüft, ob die Rechte zum Bearbeiten einer Datei ausreichen"""
    erforderliche_rechte = Berechtigung.LESEN | Berechtigung.SCHREIBEN
    return (rechte & erforderliche_rechte) == erforderliche_rechte

print(kann_datei_bearbeiten(user_rechte))   # True
print(kann_datei_bearbeiten(Berechtigung.LESEN))  # False
print(kann_datei_bearbeiten(admin_rechte))  # True

print(f"{Berechtigung.LESEN in user_rechte = }")
print(f"{Berechtigung.AUSFUEHREN in user_rechte = }")

Eine weitere nützliche Variante ist das `IntFlag` Enum, das die Funktionalität von Flag Enums mit den Eigenschaften von Integern kombiniert. Der entscheidende Unterschied liegt darin, dass `IntFlag` Werte direkt mit Ganzzahlen verglichen und sortiert werden können, da sie sich wie normale Integer verhalten.

`IntFlag` eignet sich besonders gut für Situationen, in denen Sie sowohl die Kombinierbarkeit von Flags als auch die natürliche Ordnung von Zahlen benötigen. Dadurch können Sie nicht nur prüfen, ob bestimmte Flags gesetzt sind, sondern auch die Enum-Werte sortieren oder direkt mit Zahlen vergleichen.

Die Zugriffsmöglichkeiten sind vielfältig: Sie können über den Namen (`Weekday.MONDAY`), den Wert (`Weekday(1)`) oder sogar über String-Indizierung (`Weekday["MONDAY"]`) auf die Enum-Werte zugreifen. Die Kombination mit bitweisen Operatoren bleibt dabei vollständig erhalten.

In [ ]:
from enum import IntFlag, auto

class Weekday(IntFlag):
    MONDAY = auto()
    TUESDAY = auto()
    WEDNESDAY = auto()
    THURSDAY = auto()
    FRIDAY = auto()
    SATURDAY = auto()
    SUNDAY = auto()


In [ ]:

print(Weekday.MONDAY)
print(Weekday.TUESDAY.name)
print(Weekday.WEDNESDAY.value)

print(Weekday(1))
print(Weekday["MONDAY"])



In [ ]:

print(Weekday.MONDAY == Weekday.TUESDAY)  # False
print(Weekday.MONDAY == Weekday.MONDAY)  # True
print(f"{Weekday.MONDAY < Weekday.TUESDAY = }")  # True
print(f"{Weekday.MONDAY > Weekday.TUESDAY = }")  # False


In [ ]:

print(sorted(
    [Weekday.MONDAY, 
    Weekday.WEDNESDAY, 
    Weekday.TUESDAY, 
    Weekday.FRIDAY, 
    Weekday.THURSDAY, 
    Weekday.SATURDAY]
    
))

weekend = Weekday.SATURDAY | Weekday.SUNDAY
Weekday.SATURDAY in weekend  # True
Weekday.MONDAY in weekend  # False





Die Kombination von Dataclasses und Enums zeigt die wahre Stärke beider Konzepte: Während Dataclasses eine saubere Struktur für Datenklassen bieten, sorgen Enums für typsichere und aussagekräftige Konstanten. Zusammen ergeben sie eine elegante Lösung für datenorientierte Klassen mit kontrollierten Zuständen.

In diesem Beispiel nutzen wir ein Enum als Typ-Annotation für ein Dataclass-Attribut und definieren gleichzeitig einen Standardwert. Dies bietet mehrere Vorteile: Der Code wird selbstdokumentierend, da die möglichen Status-Werte klar ersichtlich sind. IDEs können Autovervollständigung anbieten und Typfehler bereits vor der Laufzeit erkennen. Darüber hinaus verhindert das Enum ungültige Zuweisungen - es ist unmöglich, versehentlich einen String wie "active" statt "aktiv" zu verwenden.

Die Methoden der Dataclass können das Enum für Zustandsübergänge und -validierungen nutzen, wodurch die Geschäftslogik klar und wartbar wird. Diese Kombination ist besonders wertvoll bei komplexeren Domänenmodellen, wo sowohl strukturierte Daten als auch kontrollierte Zustände wichtig sind.

In [ ]:
from enum import Enum
from dataclasses import dataclass
from typing import Optional

class Status(Enum):
    AKTIV = "aktiv"
    INAKTIV = "inaktiv"
    GESPERRT = "gesperrt"

@dataclass
class Benutzer:
    name: str
    email: str
    status: Status = Status.AKTIV  # Enum als Typ-Annotation und Standardwert
    
    def aktivieren(self):
        """Aktiviert den Benutzer, wenn er inaktiv oder gesperrt ist"""
        if self.status in [Status.INAKTIV, Status.GESPERRT]:
            self.status = Status.AKTIV
            print(f"{self.name} wurde aktiviert")
        else:
            print(f"{self.name} ist bereits aktiv")



In [ ]:

benutzer = Benutzer("Max Mustermann", "max@example.com", Status.INAKTIV)
benutzer.aktivieren()  # Max Mustermann wurde aktiviert
benutzer.aktivieren()

### Zusammenfassung
Enums sind ein mächtiges Werkzeug, das Ihren Python-Code robuster, lesbarer und wartbarer macht. Sie bieten eine elegante Lösung für das häufige Problem, zusammengehörige Konstanten zu verwalten. Durch die verschiedenen Enum-Typen (Enum, IntEnum, Flag) können Sie das Verhalten genau an Ihre Bedürfnisse anpassen.


## Collections

Dokumentation: [Collections](https://docs.python.org/3/library/collections.html)

Die `collections`-Bibliothek in Python bietet spezialisierte Container-Datentypen, die über die eingebauten Datentypen wie Listen, Tupel und Dictionaries hinausgehen. Diese erweiterten Datentypen sind optimiert für bestimmte Anwendungsfälle und bieten zusätzliche Funktionalitäten, die die Arbeit mit Datenstrukturen erleichtern.

### `NamedTuple`

`NamedTuple` wird verwendet, um tuple-ähnliche Objekte mit benannten Feldern zu erstellen. Es kombiniert die Effizienz und Unveränderlichkeit von Tupeln mit einer verbesserten Lesbarkeit durch benannte Attribute.

In [ ]:
person = ('Alice', 30, 'Engineer')  # Name, Alter, Beruf
print(f"Name: {person[0]}, Age: {person[1]}, Job: {person[2]}")

In [ ]:
from collections import namedtuple

Person = namedtuple('Person', ['name', 'age', 'job'])


In [ ]:

person = Person(name='Alice', age=30, job='Engineer')
print(f"Name: {person.name}, Age: {person.age}, Job: {person.job}")
print(f"{person = }")
print(f"{person[0] = }, {person[1] = }, {person[2] = }") # Accessing by index
print(f"{person._asdict() = }")  # Convert to dictionary
print(f"{type(person) = }")  # Type of the namedtuple
print(f"{isinstance(person, tuple) = }") # namedtuple is a subclass of tuple
print(f"{Person.mro() = }")


### `defaultdict`

Ein `defaultdict` ist eine Erweiterung des normalen Dictionaries, das automatisch eine Standardwertfunktion aufruft, wenn auf einen fehlenden Schlüssel zugegriffen wird. Dies kann das handhaben von Schlüssel-Fehlern vereinfachen und den Code sauberer machen.


In [ ]:

word_counts = {}
words = ['apple', 'banana', 'apple', 'orange', 'banana', 'banana']

for word in words:
    if word in word_counts:
        word_counts[word] += 1
    else:
        word_counts[word] = 1

print(word_counts)


In [ ]:
from collections import defaultdict

word_counts = defaultdict(int)
words = ['apple', 'banana', 'apple', 'orange', 'banana', 'banana']

for word in words:
    word_counts[word] += 1

print(word_counts)


### `Counter`

`Counter` ist ein spezieller Typ von Dictionary, der hauptsächlich für das Zählen von Hash-Objekten geeignet ist. Er vereinfacht das Berechnen der Frequenz von Elementen in einer Sammlung stark.



In [ ]:

word_counts = {}
words = ['apple', 'banana', 'apple', 'orange', 'banana', 'banana']

for word in words:
    if word in word_counts:
        word_counts[word] += 1
    else:
        word_counts[word] = 1

print(word_counts)



In [ ]:
from collections import Counter

words = ['apple', 'banana', 'apple', 'orange', 'banana', 'banana']
word_counts = Counter(words)

print(word_counts)



### Vorteile der Verwendung von `NamedTuple`, `defaultdict` und `Counter`:

- **`NamedTuple`**: Verbessert die Lesbarkeit und Verständlichkeit, während die Effizienz von Tupel beibehalten wird.
- **`defaultdict`**: Reduziert die Notwendigkeit für existierende Schlüsselprüfungen und initialisiert automatisch Werte.
- **`Counter`**: Bietet eine einfache und klare Möglichkeit, die Häufigkeit von Elementen zu ermitteln, ohne zusätzlichen Code zum Zählen selbst schreiben zu müssen.

Diese spezialisierteren Datentypen sind besonders nützlich, um den Code sauberer, effizienter und weniger anfällig für Fehler zu gestalten.